# Performance Comparison: Numba vs. Cython vs. Vectorbt vs. Pandas/Numpy

This notebook compares the performance of four different Python optimization techniques for a common financial data processing task:

1.  **Numba:** A just-in-time compiler for Python that translates a subset of Python and NumPy code into fast machine code.
2.  **Cython:** A static compiler for both Python and the extended Cython language. It makes it possible to write C extensions for Python.
3.  **Vectorbt:** A library for backtesting and analyzing trading strategies that is built on top of Numba and pandas.
4.  **Pandas/Numpy:** The standard libraries for data manipulation in Python.

## Scenario

The scenario involves the following steps:

1.  Load 1-minute OHLCV data from a TimescaleDB database.
2.  Resample the data to 5m, 15m, 1h, 4h, 24h, and 7d resolutions.
3.  Calculate 20 technical indicators for each resampled timeframe.
4.  Save the results back to TimescaleDB.
5.  Measure the total time taken for the entire process for each of the four cases.

## 1. Setup

In [18]:
import time
import os
import pandas as pd
import numpy as np
import numba
import psycopg
import vectorbt as vbt
import ta
from dotenv import load_dotenv
import logging

# Import the compiled Cython module
import indicators_cython

# Load environment variables from .env file
load_dotenv()

# --- Database Connection Details ---
DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_NAME = os.getenv("POSTGRES_DB")
DATABASE_URL = os.getenv("DATABASE_URL")

# --- Timeframes ---
TIMEFRAMES = ['5T']

In [2]:
DB_CONFIG = {
    "dbname": "quant_db",
    "user": "quant_user",
    "password": "quant_password", # 실제 비밀번호를 입력하세요 (Enter your actual password).
    "host": "localhost",
    "port": "5433"
}
# 분석할 자산 및 기간 (Asset and period to analyze)
TIMEFRAME = '1min' # ohlcv_1hour, ohlcv_5min etc.
SYMBOL = 'BTCUSDT' # 데이터베이스에 저장된 심볼과 일치해야 합니다 (Must match the symbol in the database).
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'

def fetch_manual_df(query: str, params: tuple, conn_string: str) -> pd.DataFrame:
    with psycopg.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query, params)
            columns = [desc.name for desc in cur.description]  # 컬럼 이름 추출
            rows = cur.fetchall()  # 모든 결과 가져오기
            df = pd.DataFrame(rows, columns=columns)
            return df

def fetch_data_from_db(timeframe: str, symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """지정된 타임프레임 테이블에서 데이터를 가져옵니다 (psycopg3 수동 방식 사용)."""
    table_name = f"ohlcv_{timeframe}"
    logging.info(f"'{table_name}' 테이블에서 데이터 가져오기 시작...")
    conn_string = f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} password={DB_CONFIG['password']} host={DB_CONFIG['host']} port={DB_CONFIG['port']}"
    
    try:
        query = f"""
            SELECT * FROM "{table_name}"
            WHERE symbol = %s AND time >= %s AND time < %s
            ORDER BY time;
        """
        params = (symbol, start_date, end_date)
        df = fetch_manual_df(query, params, conn_string)

        if not df.empty:
            logging.info(f"'{table_name}'에서 {len(df)}개의 행을 성공적으로 가져왔습니다.")
            df['time'] = pd.to_datetime(df['time'])
            df = df.set_index('time')
        else:
            logging.warning(f"'{table_name}'에서 데이터를 찾을 수 없습니다.")
        return df
    except Exception as e:
        logging.error(f"데이터베이스 오류 ({table_name}): {e}")
        return pd.DataFrame()

# --- Load Data ---
df = fetch_data_from_db(TIMEFRAME, SYMBOL, START_DATE, END_DATE)

if df.empty:
    logging.error("Data could not be loaded. Exiting script.")
    # exit() # Uncomment to stop script if data loading fails
else:
    print("Data loaded successfully from Database.")
    print(df.head())

Data loaded successfully from Database.
                            symbol      open      high       low     close  \
time                                                                         
2024-01-01 00:00:00+00:00  BTCUSDT  42283.58  42298.62  42261.02  42298.61   
2024-01-01 00:01:00+00:00  BTCUSDT  42298.62  42320.00  42298.61  42320.00   
2024-01-01 00:02:00+00:00  BTCUSDT  42319.99  42331.54  42319.99  42325.50   
2024-01-01 00:03:00+00:00  BTCUSDT  42325.50  42368.00  42325.49  42367.99   
2024-01-01 00:04:00+00:00  BTCUSDT  42368.00  42397.23  42367.99  42397.23   

                             volume  
time                                 
2024-01-01 00:00:00+00:00  35.92724  
2024-01-01 00:01:00+00:00  21.16779  
2024-01-01 00:02:00+00:00  21.60391  
2024-01-01 00:03:00+00:00  30.50730  
2024-01-01 00:04:00+00:00  46.05107  


## 2. Database Functions

In [3]:
# def load_data_from_db(table_name: str = 'ohlcv_1m'):
#     """Loads OHLCV data from the TimescaleDB."""
#     print(f"Loading data from {table_name}...")
#     try:
#         with psycopg3.connect(DATABASE_URL) as conn:
#             query = f"SELECT * FROM {table_name}"
#             df = pd.read_sql_query(query, conn, index_col='timestamp')
#             print("Data loaded.")
#             return df
#     except Exception as e:
#         print(f"Error loading data: {e}")
#         # Fallback to placeholder data if DB connection fails
#         print("Falling back to placeholder data.")
#         rng = pd.date_range('2023-01-01', periods=365*24*60, freq='T')
#         df = pd.DataFrame(index=rng)
#         df['open'] = np.random.uniform(99, 101, size=len(df))
#         df['high'] = df['open'] + np.random.uniform(0, 1, size=len(df))
#         df['low'] = df['open'] - np.random.uniform(0, 1, size=len(df))
#         df['close'] = df['open'] + np.random.uniform(-0.5, 0.5, size=len(df))
#         df['volume'] = np.random.uniform(1000, 5000, size=len(df))
#         return df

def save_data_to_db(df: pd.DataFrame, table_name: str):
    """Saves a DataFrame to a table in TimescaleDB."""
    print(f"Saving data to {table_name}...")
    try:
        # with psycopg.connect(DATABASE_URL) as conn:
        #     # This is a simplified example. A real implementation would handle table creation and updates.
        #     df.to_sql(table_name, conn, if_exists='replace', index=True)
        #     print("Data saved.")
        print(df.describe())
    except Exception as e:
        print(f"Error saving data: {e}")

## 3. Indicator Calculation Functions

### Case A: Numba + Numpy

In [10]:
@numba.jit(nopython=True)
def _calculate_indicators_numba(open, high, low, close, volume):
    # This is a simplified example. The 'ta' library cannot be used in nopython mode.
    # A real implementation would require writing the indicator logic in Numba-compatible code.
    # For this placeholder, we'll do some simple numpy operations.
    rsi = ta.momentum.RSIIndicator(close, window=14, fillna=True).rsi()
    macd = ta.trend.MACD(close, window_slow=26, window_fast=12, window_sign=9, fillna=True).macd()
    # ... and 18
    # ... and 18 more placeholders
    return rsi, macd

def calculate_indicators_numba(df: pd.DataFrame):
    """Calculates indicators using Numba.
    Note: The 'ta' library is not compatible with Numba's nopython mode.
    We will use the 'ta' library outside the jitted function for this example,
    which is a more realistic use case.
    """
    df['rsi'] = rsi
    df['macd'] = macd
    # The above line calculates many indicators. We'll just return the df.
    # The numba function is just for demonstration of the concept.
    return df

### Case B: Cython + Numpy

In [5]:
def calculate_indicators_cython(df: pd.DataFrame):
    """Calculates indicators using the compiled Cython module.
    The Cython function is a placeholder, so we'll use the 'ta' library here as well.
    """
    close = df['close']
    # The cython function is a placeholder, so we'll call it but the result is not used.
    _ = indicators_cython.calculate_indicators_cython(df.values)
    # We use the 'ta' library for the actual calculation in this example.
    rsi = ta.momentum.RSIIndicator(close, window=14).rsi()
    macd = ta.trend.MACD(close, window_slow=26, window_fast=12, window_sign=9).macd()
    
    df['rsi'] = rsi
    df['macd'] = macd
    return df

### Case C: Vectorbt

In [6]:
def calculate_indicators_vbt(df: pd.DataFrame):
    """Calculates indicators using vectorbt."""
    price = df['close']
    # Calculate a few indicators using vectorbt
    rsi = vbt.RSI.run(price, window=14).rsi
    macd = vbt.MACD.run(price).macd
    # In a real scenario, you would calculate all 20 indicators.
    # For this example, we'll just add these two to the dataframe.
    df['rsi'] = rsi
    df['macd'] = macd
    return df

### Case D: Normal Pandas + Numpy

In [7]:
def calculate_indicators_pandas(df: pd.DataFrame):
    """Calculates indicators using pure pandas and numpy."""
    rsi = ta.momentum.RSIIndicator(df['close'], window=14).rsi().values
    macd = ta.trend.MACD(df['close'], window_slow=26, window_fast=12, window_sign=9).macd().values

    df['rsi'] = rsi
    df['macd'] = macd
    return df

## 4. Performance Comparison

In [13]:
def run_performance_test():
    results = {"Case": [], "Time (s)": []}
    ohlcv_1m = df.copy()
    
    cases = {
        "Numba": calculate_indicators_numba_optimized,
        "Cython": calculate_indicators_cython,
        "Vectorbt": calculate_indicators_vbt,
        "Pandas/Numpy": calculate_indicators_pandas
    }
    
    for case_name, calc_func in cases.items():
        print(f"--- Running Case: {case_name} ---")
        start_time = time.perf_counter()
        
        for timeframe in TIMEFRAMES:
            print(f"Processing timeframe: {timeframe}")
            # Resample data
            resampled_df = ohlcv_1m.resample(timeframe).agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).dropna()
            
            # Calculate indicators
            indicators_df = calc_func(resampled_df)
            
            # Save to DB
            save_data_to_db(indicators_df, f'ohlcv_{timeframe.lower()}_indicators_{case_name.lower()}')
            
        end_time = time.perf_counter()
        elapsed_time = end_time - start_time
        results["Case"].append(case_name)
        results["Time (s)"].append(elapsed_time)
        print(f"--- {case_name} finished in {elapsed_time:.2f} seconds ---\n")
        
    return pd.DataFrame(results)

## 5. Results

In [21]:
results_list = []
for _ in range(10):
    results = run_performance_test()
    results_list.append(results)
results_df = pd.concat(results_list, keys=range(1, 11), names=['Run', 'Index']).reset_index(level='Index', drop=True).reset_index()
print("--- Performance Comparison Results ---")
print(results_df)

--- Running Case: Numba ---
Processing timeframe: 5T
Saving data to ohlcv_5t_indicators_numba...
                open           high            low          close  \
count  105120.000000  105120.000000  105120.000000  105120.000000   
mean    65819.606907   65883.720977   65753.430133   65820.086783   
std     14627.120391   14642.955075   14610.649463   14627.177346   
min     38584.330000   38657.440000   38555.000000   38584.330000   
25%     58997.775000   59058.000000   58933.817500   58997.997500   
50%     64186.350000   64247.345000   64128.585000   64186.810000   
75%     69008.000000   69072.000000   68948.755000   69008.102500   
max    108258.380000  108353.000000  107917.540000  108258.390000   

              volume            rsi           macd    macd_signal  
count  105120.000000  105106.000000  105095.000000  105087.000000  
mean      122.826295      50.573518       3.362609       3.368085  
std       163.465975      15.392712     122.190141     114.984254  
min      

/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({


Saving data to ohlcv_5t_indicators_vectorbt...
                open           high            low          close  \
count  105120.000000  105120.000000  105120.000000  105120.000000   
mean    65819.606907   65883.720977   65753.430133   65820.086783   
std     14627.120391   14642.955075   14610.649463   14627.177346   
min     38584.330000   38657.440000   38555.000000   38584.330000   
25%     58997.775000   59058.000000   58933.817500   58997.997500   
50%     64186.350000   64247.345000   64128.585000   64186.810000   
75%     69008.000000   69072.000000   68948.755000   69008.102500   
max    108258.380000  108353.000000  107917.540000  108258.390000   

              volume            rsi           macd  
count  105120.000000  105106.000000  105095.000000  
mean      122.826295      50.573518       3.341368  
std       163.465975      15.392712     172.006077  
min         2.166770       0.350697   -2347.775128  
25%        41.599740      39.700541     -63.036154  
50%        75

/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({


Saving data to ohlcv_5t_indicators_vectorbt...
                open           high            low          close  \
count  105120.000000  105120.000000  105120.000000  105120.000000   
mean    65819.606907   65883.720977   65753.430133   65820.086783   
std     14627.120391   14642.955075   14610.649463   14627.177346   
min     38584.330000   38657.440000   38555.000000   38584.330000   
25%     58997.775000   59058.000000   58933.817500   58997.997500   
50%     64186.350000   64247.345000   64128.585000   64186.810000   
75%     69008.000000   69072.000000   68948.755000   69008.102500   
max    108258.380000  108353.000000  107917.540000  108258.390000   

              volume            rsi           macd  
count  105120.000000  105106.000000  105095.000000  
mean      122.826295      50.573518       3.341368  
std       163.465975      15.392712     172.006077  
min         2.166770       0.350697   -2347.775128  
25%        41.599740      39.700541     -63.036154  
50%        75

...

/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({


Saving data to ohlcv_5t_indicators_vectorbt...
                open           high            low          close  \
count  105120.000000  105120.000000  105120.000000  105120.000000   
mean    65819.606907   65883.720977   65753.430133   65820.086783   
std     14627.120391   14642.955075   14610.649463   14627.177346   
min     38584.330000   38657.440000   38555.000000   38584.330000   
25%     58997.775000   59058.000000   58933.817500   58997.997500   
50%     64186.350000   64247.345000   64128.585000   64186.810000   
75%     69008.000000   69072.000000   68948.755000   69008.102500   
max    108258.380000  108353.000000  107917.540000  108258.390000   

              volume            rsi           macd  
count  105120.000000  105106.000000  105095.000000  
mean      122.826295      50.573518       3.341368  
std       163.465975      15.392712     172.006077  
min         2.166770       0.350697   -2347.775128  
25%        41.599740      39.700541     -63.036154  
50%        75

/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({
/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({


Saving data to ohlcv_5t_indicators_vectorbt...
                open           high            low          close  \
count  105120.000000  105120.000000  105120.000000  105120.000000   
mean    65819.606907   65883.720977   65753.430133   65820.086783   
std     14627.120391   14642.955075   14610.649463   14627.177346   
min     38584.330000   38657.440000   38555.000000   38584.330000   
25%     58997.775000   59058.000000   58933.817500   58997.997500   
50%     64186.350000   64247.345000   64128.585000   64186.810000   
75%     69008.000000   69072.000000   68948.755000   69008.102500   
max    108258.380000  108353.000000  107917.540000  108258.390000   

              volume            rsi           macd  
count  105120.000000  105106.000000  105095.000000  
mean      122.826295      50.573518       3.341368  
std       163.465975      15.392712     172.006077  
min         2.166770       0.350697   -2347.775128  
25%        41.599740      39.700541     -63.036154  
50%        75

/tmp/ipykernel_1492069/701377152.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = ohlcv_1m.resample(timeframe).agg({


In [12]:
from numba import njit
import numpy as np

@njit
def calc_rsi_numba(close, window=14):
    rsi = np.empty_like(close)
    rsi[:] = np.nan
    for i in range(window, len(close)):
        gains = 0.0
        losses = 0.0
        for j in range(i - window + 1, i + 1):
            diff = close[j] - close[j - 1]
            if diff > 0:
                gains += diff
            else:
                losses -= diff
        avg_gain = gains / window
        avg_loss = losses / window
        if avg_loss == 0:
            rsi[i] = 100.0
        else:
            rs = avg_gain / avg_loss
            rsi[i] = 100.0 - (100.0 / (1.0 + rs))
    return rsi

@njit
def calc_macd_numba(close, fast=12, slow=26, signal=9):
    ema_fast = np.empty_like(close)
    ema_slow = np.empty_like(close)
    ema_fast[:] = np.nan
    ema_slow[:] = np.nan

    k_fast = 2 / (fast + 1)
    k_slow = 2 / (slow + 1)

    ema_fast[fast-1] = np.mean(close[:fast])
    ema_slow[slow-1] = np.mean(close[:slow])

    for i in range(fast, len(close)):
        ema_fast[i] = close[i] * k_fast + ema_fast[i-1] * (1 - k_fast)
    for i in range(slow, len(close)):
        ema_slow[i] = close[i] * k_slow + ema_slow[i-1] * (1 - k_slow)

    macd = ema_fast - ema_slow
    signal_arr = np.empty_like(macd)
    signal_arr[:] = np.nan
    # Signal line
    valid_macd = macd[~np.isnan(macd)]
    if len(valid_macd) >= signal:
        signal_arr[slow+signal-2] = np.mean(valid_macd[:signal])
        for i in range(slow+signal-1, len(macd)):
            signal_arr[i] = macd[i] * (2/(signal+1)) + signal_arr[i-1] * (1 - (2/(signal+1)))
    return macd, signal_arr

# Usage example:
close_np = df['close'].values
rsi_numba = calc_rsi_numba(close_np, window=14)
macd_numba, macd_signal_numba = calc_macd_numba(close_np)

def calculate_indicators_numba_optimized(df: pd.DataFrame):
    """Calculates indicators using optimized Numba functions."""
    close = df['close'].values
    rsi = calc_rsi_numba(close, window=14)
    macd, macd_signal = calc_macd_numba(close, fast=12, slow=26, signal=9)
    
    df['rsi'] = rsi
    df['macd'] = macd
    df['macd_signal'] = macd_signal
    return df